In [1]:
#!pip install evaluate
#!pip install peft==0.10.0

from transformers import WhisperFeatureExtractor, WhisperProcessor, WhisperTokenizer, WhisperForConditionalGeneration
import evaluate

import tqdm
import peft

d:\anaconda3\envs\whisperx\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jan 18 19:25:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| 32%   51C    P0             49W /  215W |     760MiB /   8192MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
import json
from datasets import Dataset, Audio
from tqdm import tqdm
from datasets import Dataset, Audio, DatasetDict
import re

def clean_text(text):
    # Remove non-ASCII characters and extra spaces
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def create_manifest(audio_dir, text_dir, manifest_path):
    manifest = []
    for file_name in tqdm(os.listdir(audio_dir)[:5000]): # too much data, so use less
        if file_name.endswith(".wav"):
            audio_path = os.path.join(audio_dir, file_name)
            transcription_path = os.path.join(text_dir, os.path.splitext(file_name)[0] + ".txt")
            if os.path.exists(transcription_path):
                with open(transcription_path, "r") as f:
                    transcription = clean_text(f.read().strip())
                    if len(transcription.split(" ")) < 7:
                        continue
                manifest.append({
                    "audio": audio_path,
                    "transcription": transcription
                })
    with open(manifest_path, "w") as f:
        json.dump(manifest, f, indent=4)

def load_dataset(manifest_path, test_size=0.2):
    with open(manifest_path, "r") as f:
        data = json.load(f)
    dataset = Dataset.from_list(data)
    dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
            
    # Split the dataset into train and validation sets
    train_test_split = dataset.train_test_split(test_size)
    dataset = DatasetDict({
        "train": train_test_split["train"],
        "validation": train_test_split["test"]
    })
            
    return dataset

# Example usage
train_audio_dir = "data/split_audio"
train_text_dir = "data/split_text"
manifest_path = "train_manifest.json"

# Create manifest file
create_manifest(train_audio_dir, train_text_dir, manifest_path)

# Load dataset
dataset = load_dataset(manifest_path, test_size=0.2)
print(dataset)

100%|██████████| 5000/5000 [00:01<00:00, 4306.03it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2403
    })
    validation: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 601
    })
})


In [4]:
print(dataset["train"][0])

{'audio': {'path': 'data/split_audio\\20190918_FOMC_333.wav', 'array': array([ 1.22070312e-04,  1.22070312e-04,  1.22070312e-04, ...,
       -3.05175781e-05,  0.00000000e+00,  3.96728516e-04]), 'sampling_rate': 16000}, 'transcription': "the appropriate path for monetary policy in that individual person's"}


In [5]:
from transformers import WhisperFeatureExtractor, WhisperProcessor, WhisperTokenizer, WhisperForConditionalGeneration

language = "english"
model_name = "openai/whisper-small"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language=language, task="transcribe")
processor = WhisperProcessor.from_pretrained(model_name, language=language, task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_name)


def prepare_dataset(batch):
    # load audio data
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch



d:\anaconda3\envs\whisperx\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"])
dataset["train"], dataset["validation"]

Map: 100%|██████████| 601/601 [00:06<00:00, 92.06 examples/s] 


(Dataset({
     features: ['input_features', 'labels'],
     num_rows: 2403
 }),
 Dataset({
     features: ['input_features', 'labels'],
     num_rows: 601
 }))

In [7]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [8]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [9]:
import evaluate

metric = evaluate.load("wer")

In [10]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [11]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.generation_config.language = "english"

In [12]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.5, bias="none")

model = get_peft_model(model, config)

model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.442854145857274


we are only using ~1% of the trainable parameters

In [13]:
from transformers import Seq2SeqTrainingArguments, EarlyStoppingCallback
#!pip install accelerate==0.28.0

training_args = Seq2SeqTrainingArguments(
    output_dir="output_dir",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    gradient_checkpointing=False,
    fp16=True,
    evaluation_strategy="epoch",
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=128,
    num_train_epochs=5,
    save_steps=0.2,
    logging_steps=25,
    report_to=["tensorboard"],
    metric_for_best_model="loss",  # Use loss as the metric for early stopping
    greater_is_better=False,
    push_to_hub=True,
    remove_unused_columns=False,
    label_names=["labels"]
)

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor
)

d:\anaconda3\envs\whisperx\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
trainer.train()

  2%|▏         | 25/1505 [17:40<15:34:27, 37.88s/it]

{'loss': 0.9117, 'grad_norm': 1.068368673324585, 'learning_rate': 1.966777408637874e-05, 'epoch': 0.08}


  3%|▎         | 50/1505 [30:43<12:20:26, 30.53s/it]

{'loss': 0.8347, 'grad_norm': 1.4269933700561523, 'learning_rate': 1.933554817275748e-05, 'epoch': 0.17}


  5%|▍         | 75/1505 [43:46<12:26:10, 31.31s/it]

...

{'loss': 0.1279, 'grad_norm': 0.38487187027931213, 'learning_rate': 8.053156146179403e-06, 'epoch': 2.99}


 60%|██████    | 903/1505 [12:21:47<7:20:27, 43.90s/it]d:\anaconda3\envs\whisperx\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
                                                       
 60%|██████    | 903/1505 [13:49:24<7:20:27, 43.90s/it]

{'eval_loss': 0.15477199852466583, 'eval_wer': 6.210691823899371, 'eval_runtime': 5255.6277, 'eval_samples_per_second': 0.114, 'eval_steps_per_second': 0.057, 'epoch': 3.0}


 61%|██████▏   | 923/1505 [14:04:53<7:45:28, 47.99s/it]    

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("output_dir/run4")
processor.save_pretrained("output_dir/run4")

[]

In [28]:
token = "" #specify Huggingface token to upload model to the hub

In [ ]:
model.push_to_hub("ashkab/output_dir2")

d:\anaconda3\envs\whisperx\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
adapter_model.safetensors: 100%|██████████| 14.2M/14.2M [00:02<00:00, 5.12MB/s]


CommitInfo(commit_url='https://huggingface.co/ashkab/output_dir/commit/cc0f31fc429747a7523674c6958a07035a161558', commit_message='Upload model', commit_description='', oid='cc0f31fc429747a7523674c6958a07035a161558', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ashkab/output_dir', endpoint='https://huggingface.co', repo_type='model', repo_id='ashkab/output_dir'), pr_revision=None, pr_num=None)